<h1>Creation of phenopackets from tabular data (individuals in columns)</h1>
<p>We will process <a href="https://pubmed.ncbi.nlm.nih.gov/11179005/" target="__blank">Ferrante, et al. (2001) Identification of the gene for oral-facial-digital type I syndrome</a> in this notebook.
<p>pyphetools provides a convenient way of extracting HPO terms from typical tables presented in supplemental material. Typical tables can have the individuals in columns or rows. In this case, we extract data from TABLE.</p>
<p>This note shows how to work through the table and set up the pyphetools encoder. The table was not originally available in the table, but constructed using the data in the publication</p>
<p>Users can work on one column at a time and then generate a collection of <a href="https://pubmed.ncbi.nlm.nih.gov/35705716/" target="__blank">GA4GH phenopackets</a> to represent each patient included in the original supplemental material. These phenopackets can then be used for a variety of downstream applications.</p>

In [1]:
import phenopackets as php
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.8.28


<h2>Importing HPO data</h2>

In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
PMID = "PMID:11179005"
title = "Identification of the gene for oral-facial-digital type I syndrome"
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


<h2>Importing the supplemental table</h2>
<p>Here, we use the pandas library to import this file (note that the Python package called openpyxl must be installed to read Excel files with pandas, although the library does not need to be imported in this notebook). pyphetools expects a pandas DataFrame as input, and users can choose any input format available for pandas include CSV, TSV, and Excel, or can use any other method to transform their input data into a Pandas DataFrame before using pyphetools.</p>

In [3]:
df = pd.read_excel('input/PMID_11179005.xlsx')
df['Individual'] = df['(CASE)']

In [4]:
df

,(CASE),sex,age,variant,Downslanted palpebral fissures,Dolichocephaly,Facial asymmetry,Localized skin lesion,Dental anomalies,Oral cleft,...,Dry hair,Coarse hair,Alopecia,Short 2nd toe,Syndactyly,Polycystic kidney dysplasia,Polydactyly,Clinodactyly,Brachydactyly,Individual
0,1 (F),male,1,c.1303A>C,+,+,-,+,+,+,...,-,+,+,-,-,-,-,-,-,1 (F)
1,3 (F),male,3,c.312delG,-,-,-,-,-,+,...,-,-,-,+,-,+,+,NaN,-,3 (F)
2,4 (F),male,4,c.294_312delTGGTTTGGCAAAAGAAAAG,-,-,+,-,-,+,...,+,-,+,-,+,+,-,+,-,4 (F)
3,6 (S),male,6,c.121C>T,-,-,-,+,-,+,...,-,-,-,-,+,-,-,-,-,6 (S)
4,10 (S),male,10,c.1071_1078delGAAGGATG,-,-,NaN,NaN,-,+,...,-,NaN,NaN,-,+,-,-,-,-,10 (S)
5,27 (S),male,27,c.312+2del,-,-,-,+,+,+,...,-,-,-,-,-,+,-,-,-,27 (S)
6,28 (S),male,28,c.1757delG,-,-,-,+,+,+,...,-,-,+,-,-,-,-,+,+,28 (S)


<h2>Column mappers</h2>
<p>Please see the notebook "Create phenopackets from tabular data with individuals in rows" for explanations. In the following cell we create a dictionary for the ColumnMappers. Note that the code is identical except that we use the df.loc function to get the corresponding row data</p>

In [5]:
hpo_cr = parser.get_hpo_concept_recognizer()
generator = SimpleColumnMapperGenerator(df=df, observed="+", excluded="-", hpo_cr=hpo_cr)
column_mapper_d = generator.try_mapping_columns()

In [6]:
display(HTML(generator.to_html()))

Result,Columns
Mapped,Downslanted palpebral fissures; Dolichocephaly; Facial asymmetry; Localized skin lesion; Dental anomalies; Oral cleft; Brain imaging abnormality; Developmental delay; Hepatic cysts; Dry hair; Coarse hair; Alopecia; Short 2nd toe; Syndactyly; Polycystic kidney dysplasia; Polydactyly; Clinodactyly; Brachydactyly
Unmapped,(CASE); sex; age; variant; Individual


<h2>Variant Data</h2>
<p>The OFD1 variant data (HGVS transcript) is listed in the variant column.</p>

In [7]:
default_genotype = 'hemizygous'
transcript='NM_003611.2'
vvalidator = VariantValidator(genome_build="hg38", transcript=transcript)
var_d = {}
for v in df['variant'].unique():
    var = vvalidator.encode_hgvs(v)
    var_d[v] = var

varMapper = VariantColumnMapper(variant_d=var_d,
                                variant_column_name='variant', 
                                default_genotype=default_genotype)

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003611.2%3Ac.1303A>C/NM_003611.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003611.2%3Ac.312delG/NM_003611.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003611.2%3Ac.294_312delTGGTTTGGCAAAAGAAAAG/NM_003611.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003611.2%3Ac.121C>T/NM_003611.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003611.2%3Ac.1071_1078delGAAGGATG/NM_003611.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003611.2%3Ac.312+2del/NM_003611.2?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_003611.2%3Ac.1757delG/NM_003611.2?content-t

<h1>Demographic data</h1>

In [8]:
ageMapper = AgeColumnMapper.by_year('age')
ageMapper.preview_column(df['age'])

,original column contents,age
0,1,P1Y
1,3,P3Y
2,4,P4Y
3,6,P6Y
4,10,P10Y
5,27,P27Y
6,28,P28Y


In [9]:
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='sex')
sexMapper.preview_column(df['sex'])

,original column contents,sex
0,male,MALE
1,male,MALE
2,male,MALE
3,male,MALE
4,male,MALE
5,male,MALE
6,male,MALE


In [11]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="Individual", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper, 
                        metadata=metadata,
                        pmid=PMID)
ofd1 = Disease(disease_id='OMIM:311200', disease_label='Orofaciodigital syndrome I')
encoder.set_disease(ofd1)

In [13]:
individuals = encoder.get_individuals()
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(ontology=hpo_ontology, cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
INFORMATION,NOT_MEASURED,12


In [14]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
1 (F) (MALE; P1Y),Orofaciodigital syndrome I (OMIM:311200),NM_003611.2:c.1303A>C (hemizygous),Downslanted palpebral fissures (HP:0000494); Dolichocephaly (HP:0000268); Localized skin lesion (HP:0011355); Abnormality of the dentition (HP:0000164); Orofacial cleft (HP:0000202); Brain imaging abnormality (HP:0410263); Global developmental delay (HP:0001263); Coarse hair (HP:0002208); Alopecia (HP:0001596); excluded: Facial asymmetry (HP:0000324); excluded: Hepatic cysts (HP:0001407); excluded: Dry hair (HP:0011359); excluded: Short 2nd toe (HP:0001885); excluded: Syndactyly (HP:0001159); excluded: Polycystic kidney dysplasia (HP:0000113); excluded: Polydactyly (HP:0010442); excluded: Clinodactyly (HP:0030084); excluded: Brachydactyly (HP:0001156)
3 (F) (MALE; P3Y),Orofaciodigital syndrome I (OMIM:311200),NM_003611.2:c.312+1del (hemizygous),Orofacial cleft (HP:0000202); Brain imaging abnormality (HP:0410263); Short 2nd toe (HP:0001885); Polycystic kidney dysplasia (HP:0000113); Polydactyly (HP:0010442); excluded: Downslanted palpebral fissures (HP:0000494); excluded: Dolichocephaly (HP:0000268); excluded: Facial asymmetry (HP:0000324); excluded: Localized skin lesion (HP:0011355); excluded: Abnormality of the dentition (HP:0000164); excluded: Global developmental delay (HP:0001263); excluded: Hepatic cysts (HP:0001407); excluded: Dry hair (HP:0011359); excluded: Coarse hair (HP:0002208); excluded: Alopecia (HP:0001596); excluded: Syndactyly (HP:0001159); excluded: Brachydactyly (HP:0001156)
4 (F) (MALE; P4Y),Orofaciodigital syndrome I (OMIM:311200),NM_003611.2:c.294_312del (hemizygous),Facial asymmetry (HP:0000324); Orofacial cleft (HP:0000202); Brain imaging abnormality (HP:0410263); Global developmental delay (HP:0001263); Hepatic cysts (HP:0001407); Dry hair (HP:0011359); Alopecia (HP:0001596); Syndactyly (HP:0001159); Polycystic kidney dysplasia (HP:0000113); Clinodactyly (HP:0030084); excluded: Downslanted palpebral fissures (HP:0000494); excluded: Dolichocephaly (HP:0000268); excluded: Localized skin lesion (HP:0011355); excluded: Abnormality of the dentition (HP:0000164); excluded: Coarse hair (HP:0002208); excluded: Short 2nd toe (HP:0001885); excluded: Polydactyly (HP:0010442); excluded: Brachydactyly (HP:0001156)
6 (S) (MALE; P6Y),Orofaciodigital syndrome I (OMIM:311200),NM_003611.2:c.121C>T (hemizygous),Localized skin lesion (HP:0011355); Orofacial cleft (HP:0000202); Syndactyly (HP:0001159); excluded: Downslanted palpebral fissures (HP:0000494); excluded: Dolichocephaly (HP:0000268); excluded: Facial asymmetry (HP:0000324); excluded: Abnormality of the dentition (HP:0000164); excluded: Hepatic cysts (HP:0001407); excluded: Dry hair (HP:0011359); excluded: Coarse hair (HP:0002208); excluded: Alopecia (HP:0001596); excluded: Short 2nd toe (HP:0001885); excluded: Polycystic kidney dysplasia (HP:0000113); excluded: Polydactyly (HP:0010442); excluded: Clinodactyly (HP:0030084); excluded: Brachydactyly (HP:0001156)
10 (S) (MALE; P10Y),Orofaciodigital syndrome I (OMIM:311200),NM_003611.2:c.1071_1078del (hemizygous),Orofacial cleft (HP:0000202); Syndactyly (HP:0001159); excluded: Downslanted palpebral fissures (HP:0000494); excluded: Dolichocephaly (HP:0000268); excluded: Abnormality of the dentition (HP:0000164); excluded: Dry hair (HP:0011359); excluded: Short 2nd toe (HP:0001885); excluded: Polycystic kidney dysplasia (HP:0000113); excluded: Polydactyly (HP:0010442); excluded: Clinodactyly (HP:0030084); excluded: Brachydactyly (HP:0001156)
27 (S) (MALE; P27Y),Orofaciodigital syndrome I (OMIM:311200),NM_003611.2:c.312+2del (hemizygous),Localized skin lesion (HP:0011355); Abnormality of the dentition (HP:0000164); Orofacial cleft (HP:0000202); Global developmental delay (HP:0001263); Polycystic kidney dysplasia (HP:0000113); excluded: Downslanted palpebral fissures (HP:0000494); excluded: Dolichocephaly (HP:0000268); excluded: Facial asymmetry (HP:0000324); excluded: Hepatic cys

In [15]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                             metadata=metadata,
                                             outdir=output_directory)

We output 7 GA4GH phenopackets to the directory phenopackets
